This notebook computes the derivative of the resilience indicator. This information is usefull for rendering the scorcards. This notebook must be run after compute_res_ind and before render_scorecards

In [17]:
%reset-f
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from res_ind_lib import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# data

In [18]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)

# computes all derivatives

In [19]:
deriv_set = np.setdiff1d( df_original.columns,
    ["pop","iso3","original_country","income_elast","avg_prod_k","axfin_p","axfin_r","v_s","gdp_pc_pp","gdp_pc_pp_nat",
     "protectionref","gdp_pc_pp_ref","rho" ])

pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)

pol_assess_set = ["dKtot","dWtot_currency"]
              
#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")

        


In [29]:
from progress_reporter import *

###MH set adds and compares

def compute_derivative(df_original,deriv_set,pol_assess_set):
    der = pd.DataFrame(index=df_original.index, columns=pd.MultiIndex.from_product([deriv_set,pol_assess_set], names=['inputs', 'outputs']))
    h=0.0001
    #loop on all data in df prior to add the results
    fx = compute_resiliences(df_original)[pol_assess_set]
    for var in deriv_set:
        try:
            progress_reporter(var)
            df_=df_original.copy(deep=True)
            df_[var]=df_[var]+h
            fxh= compute_resiliences(df_)[pol_assess_set]
            der[var] = ((fxh-fx)/(h))  # for elasticity: .mul(df_[var],axis=0)/fx
        except TypeError:
            print("no derivative for " +var)
    progress_reporter("done.")        
    return der.swaplevel('inputs', 'outputs', axis=1).sort_index(axis=1).dropna(how="all",axis=1)

In [38]:
derivatives = compute_derivative(df_original,deriv_set,pol_assess_set)
derivatives

Currently working on:  done.


outputs                   dKtot                                 \
inputs              T_rebuild_K              fap           far   
province                                                         
Abra                          0    325244.116552  1.313417e+06   
Agusan Del Norte              0   2301569.185141  5.901868e+06   
Agusan Del Sur                0   2850362.253591  4.186554e+06   
Aklan                         0   6992174.118631  2.088415e+07   
Albay                         0  22951469.770552  4.555687e+07   
Antique                       0   8168479.703331  1.994132e+07   
Apayao                        0    362686.037665  1.147175e+06   
Aurora                        0   1496797.984128  8.220885e+06   
Basilan                       0   7466009.696885  1.386229e+07   
Bataan                        0   2679634.519300  5.094914e+07   
Batanes                     NaN              NaN           NaN   
Batangas                      0   3399545.764583  3.252422e+07   
Benguet                       0   1476855.894191  6.178953e+07   
Biliran                       0   2496155.172766  1.133449e+07   
Bohol                         0  19057999.716339  4.298574e+07   
Bukidnon                      0   3853902.216705  8.602997e+06   
Bulacan                       0   3601031.788532  6.133509e+07   
Cagayan                       0   1827332.320672  1.067247e+07   
Camarines Norte               0   8663950.254801  2.187324e+07   
Camarines Sur                 0   4455705.147721  8.405707e+06   
Camiguin                      0   1251431.521802  2.862448e+06   
Capiz                         0   1244302.502589  5.100518e+06   
Catanduanes                   0   4357248.665416  1.480062e+07   
Cavite                        0   6339878.266659  2.564619e+08   
Cebu                          0  46719119.489889  2.716761e+08   
Compostela Valley             0   9657369.391956  2.181375e+07   
Davao Del Norte               0   3307457.798856  9.011500e+06   
Davao Del Sur                 0  25810279.253996  1.251780e+08   
Davao Oriental                0   7866963.059127  1.019560e+07   
Dinagat Islands             NaN              NaN           NaN   
...                         ...              ...           ...   
Negros Oriental             NaN              NaN           NaN   
North Cotabato                0   5854236.702311  8.586157e+06   
Northern Samar                0  11791469.636362  1.826727e+07   
Nueva Ecija                   0   2657510.774094  1.587526e+07   
Nueva Vizcaya                 0    457406.740534  3.158934e+06   
Occidental Mindoro            0   9169175.996439  2.121124e+07   
Oriental Mindoro              0  12738658.848257  3.900923e+07   
Palawan                       0  14940978.393082  5.059142e+07   
Pampanga                      0   2235660.582106  3.445305e+07   
Pangasinan                    0   2378425.304499  1.428857e+07   
Quezon                        0   1473524.531798  9.305726e+06   
Quirino                       0   1389715.385327  8.808766e+06   
Rizal                         0    990142.526161  2.468310e+07   
Romblon                       0   5853215.168752  8.527315e+06   
Samar                         0   2457079.461950  3.724176e+06   
Sarangani                     0    841207.053247  8.991564e+05   
Siquijor                    NaN              NaN           NaN   
Sorsogon                      0  13774744.946710  2.426162e+07   
South Cotabato                0   6261903.082276  2.505586e+07   
Southern Leyte                0   7519689.858276  1.304276e+07   
Sultan Kudarat                0   4132346.242665  5.028796e+06   
Sulu                          0  15212245.891347  1.729349e+07   
Surigao Del Norte             0   7914862.553563  1.408223e+07   
Surigao Del Sur             NaN              NaN           NaN   
Tarlac                        0   1341122.392684  1.112118e+07   
Tawi-Tawi                     0   5839421.162892  1.415545e+07   
Zambales                      0   422043

# output

output signs in excel

In [39]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
writer= pd.ExcelWriter("results/derivatives.xlsx", engine='xlsxwriter')
workbook=writer.book
# Add a format. Light red fill with dark red text.
red = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

blue = workbook.add_format({'bg_color': '#92c5de',
                               'font_color': '#000061'})
for outname in pol_assess_set:
    sheetname = outname[0:30]
    (derivatives[outname].dropna()).to_excel(writer,sheet_name=sheetname)
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '>',
                                    'value':    0,
                                    'format':   blue})
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '<',
                                    'value':    0,
                                    'format':   red})
    writer.sheets[sheetname].freeze_panes(1, 1)


In [43]:
derivatives

outputs                   dKtot                                 \
inputs              T_rebuild_K              fap           far   
province                                                         
Abra                          0    325244.116552  1.313417e+06   
Agusan Del Norte              0   2301569.185141  5.901868e+06   
Agusan Del Sur                0   2850362.253591  4.186554e+06   
Aklan                         0   6992174.118631  2.088415e+07   
Albay                         0  22951469.770552  4.555687e+07   
Antique                       0   8168479.703331  1.994132e+07   
Apayao                        0    362686.037665  1.147175e+06   
Aurora                        0   1496797.984128  8.220885e+06   
Basilan                       0   7466009.696885  1.386229e+07   
Bataan                        0   2679634.519300  5.094914e+07   
Batanes                     NaN              NaN           NaN   
Batangas                      0   3399545.764583  3.252422e+07   
Benguet                       0   1476855.894191  6.178953e+07   
Biliran                       0   2496155.172766  1.133449e+07   
Bohol                         0  19057999.716339  4.298574e+07   
Bukidnon                      0   3853902.216705  8.602997e+06   
Bulacan                       0   3601031.788532  6.133509e+07   
Cagayan                       0   1827332.320672  1.067247e+07   
Camarines Norte               0   8663950.254801  2.187324e+07   
Camarines Sur                 0   4455705.147721  8.405707e+06   
Camiguin                      0   1251431.521802  2.862448e+06   
Capiz                         0   1244302.502589  5.100518e+06   
Catanduanes                   0   4357248.665416  1.480062e+07   
Cavite                        0   6339878.266659  2.564619e+08   
Cebu                          0  46719119.489889  2.716761e+08   
Compostela Valley             0   9657369.391956  2.181375e+07   
Davao Del Norte               0   3307457.798856  9.011500e+06   
Davao Del Sur                 0  25810279.253996  1.251780e+08   
Davao Oriental                0   7866963.059127  1.019560e+07   
Dinagat Islands             NaN              NaN           NaN   
...                         ...              ...           ...   
Negros Oriental             NaN              NaN           NaN   
North Cotabato                0   5854236.702311  8.586157e+06   
Northern Samar                0  11791469.636362  1.826727e+07   
Nueva Ecija                   0   2657510.774094  1.587526e+07   
Nueva Vizcaya                 0    457406.740534  3.158934e+06   
Occidental Mindoro            0   9169175.996439  2.121124e+07   
Oriental Mindoro              0  12738658.848257  3.900923e+07   
Palawan                       0  14940978.393082  5.059142e+07   
Pampanga                      0   2235660.582106  3.445305e+07   
Pangasinan                    0   2378425.304499  1.428857e+07   
Quezon                        0   1473524.531798  9.305726e+06   
Quirino                       0   1389715.385327  8.808766e+06   
Rizal                         0    990142.526161  2.468310e+07   
Romblon                       0   5853215.168752  8.527315e+06   
Samar                         0   2457079.461950  3.724176e+06   
Sarangani                     0    841207.053247  8.991564e+05   
Siquijor                    NaN              NaN           NaN   
Sorsogon                      0  13774744.946710  2.426162e+07   
South Cotabato                0   6261903.082276  2.505586e+07   
Southern Leyte                0   7519689.858276  1.304276e+07   
Sultan Kudarat                0   4132346.242665  5.028796e+06   
Sulu                          0  15212245.891347  1.729349e+07   
Surigao Del Norte             0   7914862.553563  1.408223e+07   
Surigao Del Sur             NaN              NaN           NaN   
Tarlac                        0   1341122.392684  1.112118e+07   
Tawi-Tawi                     0   5839421.162892  1.415545e+07   
Zambales                      0   422043

In [41]:
import warnings
warnings.filterwarnings("always",category=UserWarning)
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")


# Double checks the results

In [33]:
#Signs of resilience derivative
for out in pol_assess_set:
    der =     np.sign(derivatives[out]).replace(0,np.nan)
    signs= pd.Series(index=der.columns)
    for i in signs.index:
        if (der[i].min()==der[i].max()): #all nonnan signs are equal
            signs[i]=der[i].min()
        else:
            print("ambigous sign for "+i+" on "+out)
            signs[i]=np.nan



ambigous sign for T_rebuild_K on dKtot
ambigous sign for sigma_p on dKtot
ambigous sign for sigma_r on dKtot
ambigous sign for social_p on dKtot
ambigous sign for social_r on dKtot
